In [1]:
# import libraries to scrape websites
import pandas as pd
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from time import sleep
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import numpy as np

In [2]:
# read the website
driver = webdriver.Chrome()
website = "https://unfccc.int/process-and-meetings/conferences/past-conferences/un-climate-change-conference-december-2019/speeches-and-statements-at-cop-25"
driver.get(website)

In [3]:
# scrape links for Statements given at official opening ceremony of the conference
links = driver.find_elements(By.XPATH, '//td[@style="height:181px; width:1128px"]/a | //td[@style="height:173px; width:1128px"]/p/a | //td[@style="height:194px; width:1128px"]/p/a')
links_list = []
for link in links:
    links_list.append(link.get_attribute('href'))

In [4]:
# scrape list of speeches
speeches = driver.find_elements(By.XPATH, '//td[@style="height:181px; width:1128px"]/a | //td[@style="height:173px; width:1128px"]/p/a | //td[@style="height:194px; width:1128px"]/p/a')
speeches_list = []
for speech in range(len(speeches)):
    speeches_list.append(speeches[speech].text)

In [5]:
# store data in to a dataframe
data = list(zip(speeches_list, links_list))
columns = ['Speech', 'Link']
df1 = pd.DataFrame(data, columns=columns)
#df['Category'] = 'Statements given at official opening ceremony of the conference'

In [6]:
# scrape links for Statements given at the opening of the high-level segment
links = driver.find_elements(By.XPATH, '//div[@class="field field--name-field-text-of-accordion-entry field--type-text-long field--label-hidden field--item"]/p/a')
links_list = []
for link in links:
    links_list.append(link.get_attribute('href'))

In [7]:
# scrape list of speeches
speeches = driver.find_elements(By.XPATH, '//div[@class="field field--name-field-text-of-accordion-entry field--type-text-long field--label-hidden field--item"]/p/a')
speeches_list = []
for speech in range(len(speeches)):
    speeches_list.append(speeches[speech].text)

In [8]:
# store data in to a dataframe
data = list(zip(speeches_list, links_list))
columns = ['Speech', 'Link']
df2 = pd.DataFrame(data, columns=columns)
df2 = df2.drop(index=range(5,9))
#df2['Category'] = 'Statements given at the opening of the high-level segment'

In [9]:
# scrape links for Statements from Heads of State and Government, ministers and heads of delegations at high-level segment
links = driver.find_elements(By.XPATH, '//td[@style="width:159px"]/a | //td[@style="width:161px"]/a | //td[@style="width:180px"]/a')
links_list = []
for link in links:
    links_list.append(link.get_attribute('href'))

In [10]:
# scrape list of speeches
speeches = driver.find_elements(By.XPATH, '//td[@style="width:159px"]/a | //td[@style="width:161px"]/a | //td[@style="width:180px"]/a')
speeches_list = []
for speech in range(len(speeches)):
    speeches_list.append(speeches[speech].text)

In [11]:
data = list(zip(speeches_list, links_list))
columns = ['Speech', 'Link']
df3 = pd.DataFrame(data, columns=columns)

In [12]:
df = pd.concat([df1, df2, df3], axis=0)

In [13]:
df

,Speech,Link
0,"Mr. Hoesung Lee, Chair of the Intergovernmenta...",https://unfccc.int/sites/default/files/resourc...
1,"H.E. Mr. Sebastián Piñera, President of the Re...",https://youtu.be/3fQHpXkI-vc?t=998
2,"H.E. Mr. António Guterres, Secretary General o...",https://unfccc.int/sites/default/files/resourc...
3,"H.E. Mr. Pedro Sánchez, President of the Gover...",https://unfccc.int/sites/default/files/resourc...
4,"Mr. Michał Kurtyka, President of COP 24/CMP 14...",https://unfccc.int/sites/default/files/resourc...
...,...,...
103,Trade Unions Non-Governmental Organizations,https://unfccc.int/documents/204778
104,"Environmental Non-Governmental Organizations, ...",https://unfccc.int/documents/204789
105,Faith-based Non-Governmental Organizations,https://unfccc.int/documents/204779
106,Research and Independent Non-Governmental Orga...,https://unfccc.int/documents/204790


In [ ]:
# reset index first
# df.reset_index(drop=True)
df.to_excel('UNFCC COP 25 Links.xlsx')

In [34]:
df = pd.read_excel('UNFCC COP 25 Links.xlsx', index_col=0)

In [37]:
# drop the youtube link
df = df.drop(1).reset_index(drop=True)

In [39]:
df['Content'] = ''

In [44]:
# retain links that direct to unfcc
unfccdf = df.drop(index=range(0,9))

# drop cuba and IGAD due to some issues
unfccdf = unfccdf.drop([69, 110], axis=0)

In [60]:
# download the pdfs
for link in unfccdf['Link']:
    driver = webdriver.Chrome()
    driver.get(link)
    
    download = WebDriverWait(driver, 0).until(EC.element_to_be_clickable((By.XPATH, '//div[@class="chosen-container chosen-container-single form-select form-control"]/a')))
    download.click()
    
    pdfdownload = WebDriverWait(driver, 0).until(EC.element_to_be_clickable((By.XPATH, '//li[@class="active-result highlighted"]')))
    pdfdownload.click()
    
    sleep(2)

In [63]:
# import libraries to convert pdfs to text
import io
from io import StringIO
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfpage import PDFPage
import os
import sys, getopt

In [64]:
#converts pdf, returns its text content as a string
def convert(fname, pages=None):
    if not pages:
        pagenums = set()
    else:
        pagenums = set(pages)

    output = io.StringIO()
    manager = PDFResourceManager()
    converter = TextConverter(manager, output, laparams=LAParams())
    interpreter = PDFPageInterpreter(manager, converter)

    infile = open(fname, 'rb')
    for page in PDFPage.get_pages(infile, pagenums):
        interpreter.process_page(page)
    infile.close()
    converter.close()
    text = output.getvalue()
    output.close
    return text 

#converts all pdfs in directory pdfDir, saves all resulting txt files to txtdir
def convertMultiple(pdfDir, txtDir):
    if pdfDir == "": pdfDir = os.getcwd() + "\\" #if no pdfDir passed in 
    for pdf in os.listdir(pdfDir): #iterate through pdfs in pdf directory
        fileExtension = pdf.split(".")[-1]
        if fileExtension == "pdf":
            pdfFilename = pdfDir + pdf 
            text = convert(pdfFilename) #get string of text content of pdf
            textFilename = txtDir + pdf + ".txt"
            textFile = open(textFilename, "w", encoding="utf-8") #make text file
            textFile.write(text) #write text to text file

In [65]:
pdfDir = "C:/Users/Documents/UNFCC COP25 PDF/"
txtDir = "C:/Users/Documents/UNFCC COP25 Text/"
convertMultiple(pdfDir, txtDir)